## Initial Parameters

In [1]:
# Define base URL for OpenAI API
base_url = "https://api.openai.com/v1/"

# Define your API key (replace with your own)
api_key = input()

# Define headers for authorization and organization
headers = {
    "Authorization": f"Bearer {api_key}",
    "OpenAI-Organization": None # replace with your own
}

#model_name = "gpt-4-0314"
#model_name = "gpt-3.5-turbo-0301"
#model_name = "gpt-3.5-turbo"
model_name = "gpt-4"

# Phase B

Phase B -> in Question + Article list + Snippets -> Ideal Answer + Exact Answers
-> Ideal Answer generator 
-> Exact Answer generator

Phase B Rules:
-> Do not return synonyms
-> Provided Articles should be enough
-> Additional Articles can be used


## Phase B Run


In [ ]:
import json
import datetime
import time  # Import the time module
import requests
import string


def append_to_logfile(logfile_name, text):
    with open(logfile_name, 'a') as logfile:
        logfile.write(text + "\n")

def remove_punctuation_and_lowercase(text):
    # Lowercase the string
    text = text.lower()

    # Remove punctuation
    text = text.translate(str.maketrans("", "", string.punctuation))

    return text

def generate_exact_answer(question, articles, snippets):
    # stub function for generating exact answer
    exact_answer = []
    if question["type"] == "yesno":
        # Generate yes/no answer
        # the exact answer of each participating system will have to be either "yes" or "no".
        params_chat = {
            "model": model_name, # model name
            "messages": [
                {"role": "system", "content": "You are BioASQ-GPT, an AI expert in question answering, \
                 research, and information retrieval in the biomedical domain."},
                {"role": "user", "content": f" {snippets}\n\n\
                 '{question['body']}'. \
                 You *must answer* only with lowercase 'yes' or 'no' even if you are not sure about the answer."}
            ],
            "temperature": 0.0, # randomness of completion
            "frequency_penalty": 0.5, # discourage repetition of words or phrases
            "presence_penalty": 0.3, # discourage new topics or entities
        }
        print(params_chat)

        # Make request to chat completion
        response_chat = requests.post(base_url + "chat/completions", headers=headers, json=params_chat)
         # Check status code
        if response_chat.status_code == 200:
            # Parse response as JSON
            data_chat = response_chat.json()

            # Get the generated message from the chat completion result
            generated_message = data_chat["choices"][0]["message"]["content"]
            print("\ngpt response yesno:")
            print(generated_message)
            generated_message = remove_punctuation_and_lowercase(generated_message)
            exact_answer = generated_message
        else:
            raise Exception(f"Error: {response_chat.status_code}")
    elif question["type"] == "factoid":
        # Generate factoid answer
        # each participating system will have to return a json string array of up to 5 entity names (e.g., up to 5 names of drugs), numbers, or similar short expressions, ordered by decreasing confidence.
        params_chat = {
            "model": model_name, # model name
            "messages": [
                {"role": "system", "content": "You are BioASQ-GPT, an AI expert in question answering, \
                 research, and information retrieval in the biomedical domain."},
                {"role": "user", "content": f" {snippets}\n\n\
                 '{question['body']}'. \
                 Answer this question by returning only a JSON string array of entity names, numbers, or similar short expressions that are an answer to the question, ordered by decreasing confidence. \
                 The array should contain at max 5 elements but can contain less. If you don't know any answer return an empty list. Return only this list, it must not contain phrases and **must be valid JSON**."}
            ],
            "temperature": 0.0, # randomness of completion
            "frequency_penalty": 0.5, # discourage repetition of words or phrases
            "presence_penalty": 0.1, # discourage new topics or entities
        }
        print(params_chat)

        # Make request to chat completion
        response_chat = requests.post(base_url + "chat/completions", headers=headers, json=params_chat)
         # Check status code
        if response_chat.status_code == 200:
            # Parse response as JSON
            data_chat = response_chat.json()

            # Get the generated message from the chat completion result
            generated_message = data_chat["choices"][0]["message"]["content"]
            print("\ngpt response factoid:")
            print(generated_message)
            
            # Parse Json
            # Extract the factoids from the generated message
            factoids = json.loads(generated_message)
            
            wrapped_list = [[item] for item in factoids]
            
            exact_answer = wrapped_list
        else:
            raise Exception(f"Error: {response_chat.status_code}")
    elif question["type"] == "list":
        # Generate list answer
        # each participating system will have to return a single JSON string array of entity names, numbers, or similar short expressions, jointly taken to constitute a single answer (e.g., the most common symptoms of a disease). 
        # The returned list will have to contain no more than 100 entries of no more than 100 characters each.
        params_chat = {
            "model": model_name, # model name
            "messages": [
                {"role": "system", "content": "You are BioASQ-GPT, an AI expert in question answering, \
                 research, and information retrieval in the biomedical domain."},
                {"role": "user", "content": f" {snippets}\n\n\
                 '{question['body']}'. \
                 Answer this question by only returning a JSON string array of entity names, numbers, or similar short expressions that are an answer to the question (e.g., the most common symptoms of a disease). \
                 The returned list will have to contain no more than 100 entries of no more than 100 characters each. If you don't know any answer return an empty list. Return only this list, it must not contain phrases and **must be valid JSON**."}
            ],
            "temperature": 0.0, # randomness of completion
            "frequency_penalty": 0.5, # discourage repetition of words or phrases
            "presence_penalty": 0.1, # discourage new topics or entities
        }
        print(params_chat)

        # Make request to chat completion
        response_chat = requests.post(base_url + "chat/completions", headers=headers, json=params_chat)
         # Check status code
        if response_chat.status_code == 200:
            # Parse response as JSON
            data_chat = response_chat.json()

            # Get the generated message from the chat completion result
            generated_message = data_chat["choices"][0]["message"]["content"]
            print("\ngpt response list answer:")
            print(generated_message)
            
            # Parse Json
            # Extract the factoids from the generated message
            list_answer = json.loads(generated_message)
            
            wrapped_list = [[item] for item in list_answer]
            
            exact_answer = wrapped_list
        else:
            raise Exception(f"Error: {response_chat.status_code}")
    return exact_answer

def generate_ideal_answer(question, articles, snippets):
    # stub function for generating ideal answer
    # a single paragraph-sized text ideally summarizing the most relevant information from articles and snippets
    # The maximum allowed length of each "ideal" answer is 200 words.
    # Each returned "ideal" answer is intended to approximate a short text that a biomedical expert would write to answer the corresponding question (e.g., including prominent supportive information)
    params_chat = {
        "model": model_name, # model name
        "messages": [
            {"role": "system", "content": "You are BioASQ-GPT, an AI expert in question answering, \
             research, and information retrieval in the biomedical domain."},
            {"role": "user", "content": f" {snippets}\n\n\
             '{question['body']}'. \
             Answer this question by returning a single paragraph-sized text ideally summarizing the most relevant information \
             The maximum allowed length of the answer is 200 words. \
             The returned answer is intended to approximate a short text that a biomedical expert would write to answer the corresponding question (e.g., including prominent supportive information)."}
        ],
        "temperature": 0.0, # randomness of completion
        "frequency_penalty": 0.5, # discourage repetition of words or phrases
        "presence_penalty": 0.7, # discourage new topics or entities
    }
    print(params_chat)


    # Make request to chat completion
    response_chat = requests.post(base_url + "chat/completions", headers=headers, json=params_chat)
     # Check status code
    if response_chat.status_code == 200:
        # Parse response as JSON
        data_chat = response_chat.json()

        # Get the generated message from the chat completion result
        generated_message = data_chat["choices"][0]["message"]["content"]
        print("\ngpt response ideal anwer:")
        print(generated_message)
        return generated_message
    else:
        raise Exception(f"Error: {response_chat.status_code}")
    

# Get the current timestamp in a sortable format
timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

logfile_name = f"{timestamp}_{model_name}_Grounded_PhaseB_log_file.json"

# Load the input file in JSON format
with open('./BioASQ-task11bPhaseB-testset4.json', encoding='utf-8') as input_file:
    data = json.loads(input_file.read())

# Create an empty list to store the results
results = []

# Iterate over all questions
offset = 0
for idx, question in enumerate(data["questions"]):
    print(f"\n\n{idx}")
    if idx < offset:
        continue
    # Determine the type of question
    question_type = question["type"]
    print(f"{question['body']}\n")

    # Get the relevant articles and snippets
    relevant_articles = question["documents"]
    relevant_snippets = question["snippets"]
    
    # Ungrounded runs
    relevant_snippets = []


     # Generate the exact answer and ideal answer
    try:
        exact_answer = generate_exact_answer(question, relevant_articles, relevant_snippets)
        ideal_answer = generate_ideal_answer(question, relevant_articles, relevant_snippets)
    except Exception as e:
        print(f"Error processing question {idx}: {e}")
        time.sleep(5)  # Sleep for 5 seconds before retrying
        exact_answer = generate_exact_answer(question, relevant_articles, relevant_snippets)
        ideal_answer = generate_ideal_answer(question, relevant_articles, relevant_snippets)

    # Create a dictionary to store the results for this question
    question_results = {
        "id": question["id"],
        "type": question_type,
        "body": question["body"],
        "ideal_answer": ideal_answer,
        "exact_answer": exact_answer,
        "documents": relevant_articles,
        "snippets": relevant_snippets
    }
    
    # Add to logfile to continue after error with offset
    append_to_logfile(logfile_name, json.dumps(question_results))

    # Add the results for this question to the list of all results
    results.append(question_results)

# Create a dictionary to store the results for all questions
output = {
    "questions": results
}

# Get the current timestamp in a sortable format
timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

# Prefix the output file name with the timestamp
output_file_name = f"{timestamp}_{model_name}_Grounded_PhaseB_output_file.json"

# Save the output to a file in pretty-formatted JSON format
with open(f"./Result/{output_file_name}", "w") as f:
    json.dump(output, f, indent=4)

